<a href="https://colab.research.google.com/github/CryptoRobotFr/video-lfdm/blob/main/live_basic_strat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ta ccxt

In [29]:
import ccxt
import pandas as pd
import numpy as np
import ta

ftx_auth_object = {
    "apiKey": "Xam6UD2WsA8rnP-B3BoszzvDuuTMIJOUmWQOfBfa",
    "secret": "O0BjYlBJu6ZLp3d500rrfh1swabyT5AdRwZgGIeL",
    'headers': {
        'FTX-SUBACCOUNT': "strat_1"
    }
}

session = ccxt.ftx(ftx_auth_object)
session.load_markets()

pair_symbol = "BTC/USDT"
symbol_coin = "BTC"
symbol_usd = "USDT"
timeframe = "1h"
limit = 1000

df = pd.DataFrame(data=session.fetch_ohlcv(
    pair_symbol, timeframe, None, limit=limit))
df = df.rename(
    columns={0: 'timestamp', 1: 'open', 2: 'high', 3: 'low', 4: 'close', 5: 'volume'})
df = df.set_index(df['timestamp'])
df.index = pd.to_datetime(df.index, unit='ms')
del df['timestamp']

df['ema1'] = ta.trend.ema_indicator(close = df['close'], window = 25) # Moyenne exponentiel courte
df['ema2'] = ta.trend.ema_indicator(close = df['close'], window = 45) # Moyenne exponentiel moyenne
df['sma_long'] = ta.trend.sma_indicator(close = df['close'], window = 600) # Moyenne simple longue
df['stoch_rsi'] = ta.momentum.stochrsi(close = df['close'], window = 14) # Stochastic RSI non moyenné (K=1 sur Trading View)

def get_balance(symbol):
    balance = 0
    try:
        balance = pd.DataFrame(session.fetchBalance())['total'][symbol]
    except:
        balance = 0
    return balance

balance_coin = get_balance(symbol_coin)
balance_usd = get_balance(symbol_usd)
row = df.iloc[-2]

if row['ema1'] > row['ema2'] and row['stoch_rsi'] < 0.8 and row['close'] > row['sma_long'] and balance_usd > 10:
    print("buy")
    amount_to_buy = balance_usd / row["close"] 
    print(session.createOrder(
                pair_symbol, 
                'market', 
                "buy", 
                session.amount_to_precision(pair_symbol, amount_to_buy),
                None
            ))
elif row['ema2'] > row['ema1'] and row['stoch_rsi'] > 0.2 and  balance_coin*row['close'] > 10:
    print("sell")
    amount_to_sell = balance_coin
    print(session.createOrder(
                pair_symbol, 
                'market', 
                "sell", 
                session.amount_to_precision(pair_symbol, amount_to_sell),
                None
            ))
else:
    print("Nothing to do just wait")



sell
{'info': {'id': '121357121708', 'clientId': None, 'market': 'BTC/USDT', 'type': 'market', 'side': 'sell', 'price': None, 'size': '0.0023', 'status': 'new', 'filledSize': '0.0', 'remainingSize': '0.0023', 'reduceOnly': False, 'liquidation': None, 'avgFillPrice': None, 'postOnly': False, 'ioc': True, 'createdAt': '2022-02-13T15:23:30.079843+00:00', 'future': None}, 'id': '121357121708', 'clientOrderId': None, 'timestamp': 1644765810079, 'datetime': '2022-02-13T15:23:30.079Z', 'lastTradeTimestamp': None, 'symbol': 'BTC/USDT', 'type': 'market', 'timeInForce': None, 'postOnly': False, 'side': 'sell', 'price': None, 'stopPrice': None, 'amount': 0.0023, 'cost': None, 'average': None, 'filled': 0.0, 'remaining': 0.0023, 'status': 'open', 'fee': None, 'trades': [], 'fees': []}
